In [2]:
import pdblp
import pandas as pd
from datetime import date
import datetime
import sys, getopt

In [3]:
con = pdblp.BCon(port=8194, timeout=5000)
con.start()

In [47]:
today = '20' + date.today().strftime('%y%m%d')
df = con.bdh('SPY US Equity', flds=['PX_LAST', 'PX_OPEN', 'MOV_AVG_200D', 'MOV_AVG_5D', 'MOV_AVG_50D'], start_date='20000101', end_date=today)

c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])


In [48]:
df

ticker     SPY US Equity                                              
field            PX_LAST   PX_OPEN MOV_AVG_200D MOV_AVG_5D MOV_AVG_50D
date                                                                  
2000-01-03      145.4375  148.2500     134.9281   146.3906    140.2524
2000-01-04      139.7500  143.5313     134.9771   145.1031    140.4455
2000-01-05      140.0000  139.9375     135.0462   143.7406    140.6568
2000-01-06      137.7500  139.6250     135.1004   141.9625    140.8555
2000-01-07      145.7500  140.3125     135.1817   141.7375    141.1680
...                  ...       ...          ...        ...         ...
2022-11-23      402.4200  399.5500     405.0620   397.4360    378.0218
2022-11-25      402.3300  401.8300     404.7859   399.0540    378.2660
2022-11-28      395.9100  399.0900     404.5189   399.0300    378.4730
2022-11-29      395.2300  396.0500     404.2927   399.1580    378.6066
2022-11-30      407.6800  395.4900     404.1360   400.7140    379.0784

[5766 rows x 5 columns]

In [49]:
window_size = 7
predictions = []
oneHotPreds = []

for i in range(len(df.values) - 1):
    row1 = df.values[i]
    row2 = df.values[i + 1]
    priceTomorrow = row2[1]
    priceToday = row1[0]

    if priceTomorrow > priceToday:
        predictions.append('UP')
        oneHotPreds.append(1)
    else:
        predictions.append('DOWN')
        oneHotPreds.append(0)

predictions.append('X')
oneHotPreds.append('X')
df['Predictions'] = predictions
df['OneHotPred'] = oneHotPreds


In [50]:
df

ticker     SPY US Equity                                                \
field            PX_LAST   PX_OPEN MOV_AVG_200D MOV_AVG_5D MOV_AVG_50D   
date                                                                     
2000-01-03      145.4375  148.2500     134.9281   146.3906    140.2524   
2000-01-04      139.7500  143.5313     134.9771   145.1031    140.4455   
2000-01-05      140.0000  139.9375     135.0462   143.7406    140.6568   
2000-01-06      137.7500  139.6250     135.1004   141.9625    140.8555   
2000-01-07      145.7500  140.3125     135.1817   141.7375    141.1680   
...                  ...       ...          ...        ...         ...   
2022-11-23      402.4200  399.5500     405.0620   397.4360    378.0218   
2022-11-25      402.3300  401.8300     404.7859   399.0540    378.2660   
2022-11-28      395.9100  399.0900     404.5189   399.0300    378.4730   
2022-11-29      395.2300  396.0500     404.2927   399.1580    378.6066   
2022-11-30      407.6800  395.4900     404.1360   400.7140    379.0784   

ticker     Predictions OneHotPred  
field                              
date                               
2000-01-03        DOWN          0  
2000-01-04          UP          1  
2000-01-05        DOWN          0  
2000-01-06          UP          1  
2000-01-07          UP          1  
...                ...        ...  
2022-11-23        DOWN          0  
2022-11-25        DOWN          0  
2022-11-28          UP          1  
2022-11-29          UP          1  
2022-11-30           X          X  

[5766 rows x 7 columns]

In [61]:
from util import loadTickers
import random 
tickers = loadTickers()
tickers.pop(0)
tickers.pop(0)
random.shuffle(tickers)
tickers.insert(0, 'SPY US Equity')

for k, ticker in enumerate(tickers):
    if k % 25 == 0:
        print('PROCESSED', k)
    
    df = con.bdh(ticker, ['PX_LAST', 'PX_OPEN', 'MOV_AVG_200D', 'MOV_AVG_5D', 'MOV_AVG_50D'], start_date='20000101', end_date=today)
    predictions = []
    oneHotPreds = []

    for i in range(len(df.values) - 1):
        row1 = df.values[i]
        row2 = df.values[i + 1]
        priceTomorrow = row2[0]
        priceToday = row1[0]

        if priceTomorrow > priceToday:
            predictions.append('UP')
            oneHotPreds.append(1)
        else:
            predictions.append('DOWN')
            oneHotPreds.append(0)

    predictions.append('X')
    oneHotPreds.append('X')
    df['Predictions'] = predictions
    df['OneHotPred'] = oneHotPreds

    fileName = '.\data\\' + ticker.split(' ')[0] + '.csv'
    df.to_csv(fileName)

PROCESSED 0


c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a

PROCESSED 25


c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a

PROCESSED 50


...

PROCESSED 175


c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a

PROCESSED 200


c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])
c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a

KeyboardInterrupt: 